In [1]:
import numpy as np
import scipy.stats
import scs

In [2]:
#############################################
#      Generate random cone problems        #
#############################################

def pos(x):
    return (x + np.abs(x)) / 2.
    
def gen_feasible(m, n, p_scale = 0.1):
    z = np.random.randn(m)
    y = pos(z)
    s = y - z

    P = np.random.randn(n,n)
    P = p_scale * P.T @ P

    # Make problem slightly more numerically challenging:
    A = np.random.randn(m, n)
    U, S, V = np.linalg.svd(A, full_matrices=False)
    S = S**2
    S /= np.max(S)
    A = (U * S) @ V
        
    x = np.random.randn(n)
    c = -A.T @ y - P @ x
    b = A.dot(x) + s
    
    #b /= np.linalg.norm(b)
    #x /= np.linalg.norm(b)
    
    #c /= np.linalg.norm(c)
    #y /= np.linalg.norm(c)
    
    return (P, A, b, c, x, y)

In [3]:
def gen_infeasible(m, n, p_scale = 0.1):
    # b'y < 0, A'y == 0
    z = np.random.randn(m)
    y = pos(z)  # y = s - z;
    
    A = np.random.randn(m, n)
    
    # A := A - y(A'y)' / y'y
    A = A - np.outer(y, np.transpose(A).dot(y)) / np.linalg.norm(y)**2

    b = np.random.randn(m)
    b = -b / np.dot(b, y)
    
    P = np.random.randn(n,n)
    P = p_scale * P.T @ P

    return (P, A, b, np.random.randn(n))

def gen_unbounded(m, n, p_scale = 0.1):
    # c'x < 0, Ax + s = 0, Px = 0
    z = np.random.randn(m)
    s = pos(z)
    
    P = np.random.randn(n,n)
    P = p_scale * P.T @ P
    eigs, V = np.linalg.eig(P)
    
    i = np.argmin(eigs)
    eigs[i] = 0
    x = V[:,i]
    
    # Px = 0
    P = (V * eigs) @ V.T
    P = 0.5 * (P + P.T)
      
    A = np.random.randn(m, n)

    # A := A - (s + Ax)x' / x'x ===> Ax + s == 0
    A = A - np.outer(s + A.dot(x), x) / np.linalg.norm(x)**2
        
    c = np.random.randn(n)
    c = -c / np.dot(c, x)
    
    return (P, A, np.random.randn(m), c)

In [4]:
def is_optimal(P, A, b, c, x, y, tol=1e-6):
    s = b - A @ x
    
    if (np.linalg.norm(P @ x + c + A.T @ y) < tol and
        np.abs(y.T @ s) < tol and
        np.linalg.norm(s - pos(s)) < tol and
        np.linalg.norm(y - pos(y)) < tol):
        return True
    return False

def is_infeasible(A, b, y, tol=1e-6):
    if b.T @ y >= 0:
        return False
    
    y_hat = y / np.abs(b.T @ y)
    if (np.linalg.norm(y_hat - pos(y_hat)) < tol and np.linalg.norm(A.T @ y_hat) < tol):
        return True
    return False

def is_unbounded(P, A, c, x, tol=1e-6):
    if c.T @ x >= 0:
        return False
    
    x_hat = x / np.abs(c.T @ x)
    if np.linalg.norm(P @ x_hat) < tol and np.linalg.norm(A @ x_hat + pos(-A @ x_hat)) < tol:
        return True
    return False

In [5]:
# ''linear'' projection logic

class LinearProjector(object):
    def __init__(self, P, A, b, c):
        (m,n) = A.shape
        self.A = A
        self.h = np.hstack((c, b))        
        self.L = scipy.linalg.cho_factor(np.eye(n) + P + A.T @ A)
        self.g = self._solve(self.h)

    def _solve(self, v):
        (m, n) = self.A.shape
        sol = np.zeros(n+m,)
        sol[:n] = scipy.linalg.cho_solve(self.L, v[:n] - self.A.T @ v[n:])
        sol[n:] = v[n:] + self.A @ sol[:n]
        return sol
        
    def project(self, w):
        g = self.g
        p = self._solve(w[:-1])
        
        _a = 1 + g.T@g
        _b = w[:-1].T @ g - 2 * p.T @ g - w[-1]
        _c = p.T@p - w[:-1].T @ p
        
        tau = (-_b + np.sqrt(_b ** 2 - 4 * _a * _c)) / 2 / _a
        
        return np.hstack((p - tau * g, tau))

# Douglas-Rachford splitting / ADMM for QP
def solve_qp_dr(P, A, b, c, N=1000, tol=1e-6):
    (m,n) = np.shape(A)

    lp = LinearProjector(P, A, b, c)

    u = np.zeros(n+m+1,)
    #u[:n+m] = np.hstack((c, b))
    u[-1] = 1.

    def proj_cone(v):
        v[n:] = np.maximum(v[n:], 0.)
        return v


    use_dr = True # slightly different DR vs ADMM
    
    for i in range(N):
        # for DR ut converges to sol *not* u, see Patrinos, Stella, Bemporad, 2014
        # v - ut go to zero
        dr_lam = 0.5 # \in [0,1], 0.5 = DR, 1.0 = PR

        ut = lp.project(u)
        v = proj_cone(2 * ut - u)
        u += 2 * dr_lam * (v - ut)
        
        x = ut[:n] / ut[-1]
        y = ut[n:-1] / ut[-1]
        
        if (is_optimal(P, A, b, c, x, y, tol=tol) or 
            is_infeasible(A, b, y, tol=tol) or
            is_unbounded(P, A, c, x, tol=tol)):
            break
        
    print(i)
    return x,y


In [6]:
m = 1500
n = 1000
N = int(5e3)
seed = 1234

In [7]:
np.random.seed(seed)
(P, A, b, c) = gen_unbounded(m, n)
#P = P + 1e-7 * np.eye(n)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=10 * N, acceleration_lookback = 10, eps=1e-6)

/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in true_divide
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in true_divide
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in matmul
  
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in matmul
  after removing the cwd from sys.path.
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in matmul
  if sys.path[0] == '':
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in matmul
  from ipykernel import kerne

4825
------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-06, alpha: 1.50, max_iters: 50000,
	  normalize: 1, scale: 1.00, adaptive_scaling: 1,
	  acceleration_lookback: 10, acceleration_interval: 50,
	  warm_start: 0
lin-sys:  sparse-direct
	  nnz(A): 1500000, nnz(P): 500500
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.27e+19  2.28e+20  1.77e+21  1.55e+21  1.00e+00  2.31e-02 
   100| 8.71e-02  1.78e+00  2.98e+00 -1.42e+02  2.10e-01  4.03e+00 
   200| 5.84e+01  9.11e-01  2.31e+01 -2.01e+04  8.29e-03  1.13e+01 
   300| 1.33e+03  9.66e-01  5.07e+02 -7.8

In [8]:
print(c.T @ sol['x'])
print(np.linalg.norm(c) * np.linalg.norm(A @ sol['x'] + sol['s']))
print(np.sqrt(max(0., sol['x'].T @ P @ sol['x'])))

-0.9999999999999991
0.00016302068657004883
6.998773443321813e-07


In [9]:
print(np.linalg.norm(P - P.T))

0.0


In [10]:
np.random.seed(seed)
(P, A, b, c) = gen_infeasible(m, n)
#P = P + np.eye(n)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6)

/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in true_divide
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in true_divide
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in matmul
  
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in matmul
  after removing the cwd from sys.path.
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in matmul
  if sys.path[0] == '':
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in matmul
  from ipykernel import kerne

518
------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-06, alpha: 1.50, max_iters: 5000,
	  normalize: 1, scale: 1.00, adaptive_scaling: 1,
	  acceleration_lookback: 10, acceleration_interval: 50,
	  warm_start: 0
lin-sys:  sparse-direct
	  nnz(A): 1500000, nnz(P): 500500
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.75e-01  4.81e+00  2.28e+00 -2.51e+00  1.00e+00  2.30e-02 
   100| 7.54e-01  4.95e+00  4.02e-01  6.90e+02  1.00e+00  8.93e-01 
   200| 2.74e+13  1.92e+14  1.09e+19  2.42e+14  1.00e+00  1.75e+00 
   300| 7.04e+10  5.99e+11  1.09e+19  5.00e

In [11]:
print(b.T @ sol['y'])
print(np.linalg.norm(b) * np.linalg.norm(A.T @ sol['y']))

-1.0000000000000007
8.069000938048694e-06


In [12]:
np.random.seed(seed)
(P, A, b, c, _x, _y) = gen_feasible(m, n, p_scale=0.1)
#(_x,_y) = solve_qp_dr(P, A, b, c, N=int())
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)

cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 5, eps=1e-9, scale=0.1)

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-09, alpha: 1.50, max_iters: 5000,
	  normalize: 1, scale: 0.10, adaptive_scaling: 1,
	  acceleration_lookback: 5, acceleration_interval: 50,
	  warm_start: 0
lin-sys:  sparse-direct
	  nnz(A): 1500000, nnz(P): 500500
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 4.13e+17  4.98e+19  8.38e+21 -9.20e+21  1.00e-01  2.23e-02 
   100| 4.02e-05  6.39e-04  4.95e-03 -4.81e+04  1.00e-01  8.68e-01 
   200| 5.44e-06  5.01e-05  9.08e-04 -4.81e+04  1.00e-01  1.72e+00 
   300| 1.49e-06  1.40e-05  1.44e-04 -4.81e+04  

In [13]:
x = sol['x']
y = sol['y']
s = sol['s']

print(_x.T@ P @ _x / 2 + c.T @ _x)
print(x.T@ P @ x / 2 + c.T @ x)

print(np.linalg.norm(x - _x))
print(np.linalg.norm(y - _y))

#x = _x
#y = _y
#s = b - A @ _x

print(np.linalg.norm(A @ x + s - b) / (1+np.linalg.norm(b)))
print(np.linalg.norm(P@x + A.T @ y + c) / (1+np.linalg.norm(c)))
print(abs(x.T@ P @ x + c.T @ x + b.T @ y) / (1 + abs(x.T@P @ x) + abs(c.T @ x) + abs(b.T @ y)))

-48121.41383246187
-48121.41383249919
3.9219615997217865e-06
0.015022402050802713
2.140565429176178e-09
1.619912428757439e-10
1.9776537255227922e-13


In [14]:
np.random.seed(seed)
(P, A, b, c, _, _) = gen_feasible(m, n, p_scale=0.)
#(x,y) = solve_qp_dr(P, A, b, c, N=N)
#print(c.T @ x)
#print(x)

In [15]:
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 0, eps=1e-6, use_indirect=False)
print(c.T @ sol['x'])

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-06, alpha: 1.50, max_iters: 5000,
	  normalize: 1, scale: 1.00, adaptive_scaling: 1,
	  acceleration_lookback: 0, acceleration_interval: 50,
	  warm_start: 0
lin-sys:  sparse-direct
	  nnz(A): 1500000, nnz(P): 500500
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 5.40e+17  2.43e+20  1.25e+21 -9.73e+21  1.00e+00  2.24e-02 
   100| 3.35e-04  1.82e+00  2.31e+01 -4.81e+04  2.88e-01  3.92e+00 
   200| 7.73e-06  4.89e-04  4.82e-03 -4.81e+04  1.51e-01  1.10e+01 
   225| 4.21e-06  1.46e-04  2.02e-03 -4.81e+04  

In [16]:
probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 0, eps=1e-6, use_indirect=True)
print(c.T @ sol['x'])

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-06, alpha: 1.50, max_iters: 5000,
	  normalize: 1, scale: 1.00, adaptive_scaling: 1,
	  acceleration_lookback: 0, acceleration_interval: 50,
	  warm_start: 0
lin-sys:  sparse-indirect
	  nnz(A): 1500000, nnz(P): 0, cg_rate: 2.00
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 3.19e+18  7.00e+17  4.82e+20 -5.01e+20  1.00e+00  4.83e+00 
   100| 6.57e-03  1.92e-03  9.69e-02  6.85e+00  1.00e+00  2.58e+01 
   200| 1.89e-03  3.20e-04  7.29e-03  6.93e+00  1.00e+00  4.14e+01 
   300| 7.02e-04  1.81e-04  6.71e-03

In [17]:
np.random.seed(seed)
(P, A, b, c, _, _) = gen_feasible(m, n, 0.001)

In [18]:
print(np.linalg.norm(A.flatten(), np.inf))
print(np.linalg.norm(P.flatten(), np.inf))

0.05248355340151854
1.1447767443167587


In [19]:
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(x)
print(y)

4999
[-1.50034703e-01 -5.03083812e-01  1.31863055e+00 -6.54483829e-01
  2.80839195e-01 -8.16171058e-01 -4.24751278e-01  6.75875471e-01
  1.26243082e+00  2.85778441e-01 -1.62700242e-01 -2.96636196e-01
 -1.14309535e+00  2.07974192e+00 -5.30883805e-01 -8.61170008e-01
  4.62746644e-01 -1.71844947e+00 -1.13915207e+00  2.64377317e-01
 -1.00971810e+00  1.54677085e-01  5.29206601e-01 -2.39448753e-01
  5.82230445e-01 -7.32849869e-01  3.48530467e-01 -1.57536586e+00
 -8.72494032e-01 -3.94790480e-01 -4.02609837e-01  2.00517308e-01
  8.95225449e-01  1.18672022e+00 -9.53873408e-01 -3.52497545e-01
 -2.35841701e+00 -1.91052193e+00 -4.80791525e-01  3.87752227e-03
  1.08792234e+00  2.49557221e-01 -9.18990895e-01  5.11760310e-01
 -8.76960677e-01 -1.65475628e-01 -4.57843511e-01  1.75070345e-01
  7.90603841e-01  2.02325205e-01  1.04695335e+00 -7.82657060e-01
  9.74851005e-01 -6.58030853e-01  1.72559815e-01 -4.43558864e-01
 -8.83038666e-01 -1.56386052e+00  1.47576991e-02  3.98694880e-01
  6.61296754e-02  4.

In [20]:
np.random.seed(seed)
(P, A, b, c, _, _) = gen_feasible(m, n)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)

cone = dict(l=m)
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 5, eps=1e-6, scale=1)

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-06, alpha: 1.50, max_iters: 5000,
	  normalize: 1, scale: 1.00, adaptive_scaling: 1,
	  acceleration_lookback: 5, acceleration_interval: 50,
	  warm_start: 0
lin-sys:  sparse-direct
	  nnz(A): 1500000, nnz(P): 500500
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 5.40e+17  2.43e+20  1.25e+21 -9.73e+21  1.00e+00  2.43e-02 
   100| 3.32e-04  1.85e+00  2.36e+01 -4.81e+04  2.91e-01  4.54e+00 
   200| 8.17e-06  4.80e-04  4.65e-03 -4.81e+04  1.51e-01  1.29e+01 
   225| 4.38e-06  1.54e-04  2.29e-03 -4.81e+04  

{'x': array([-1.49900681e-01, -5.02627706e-01,  1.31815936e+00, -6.54334308e-01,
         2.80692270e-01, -8.16195331e-01, -4.24791397e-01,  6.75663989e-01,
         1.26226484e+00,  2.85902138e-01, -1.62431745e-01, -2.96861094e-01,
        -1.14301418e+00,  2.07952526e+00, -5.30457082e-01, -8.61237742e-01,
         4.62774555e-01, -1.71826012e+00, -1.13907491e+00,  2.64211700e-01,
        -1.00975514e+00,  1.54661842e-01,  5.28977909e-01, -2.39363126e-01,
         5.82327721e-01, -7.32686886e-01,  3.48683409e-01, -1.57539766e+00,
        -8.72622271e-01, -3.94730917e-01, -4.02776091e-01,  2.00233775e-01,
         8.95197931e-01,  1.18676205e+00, -9.53928558e-01, -3.52752021e-01,
        -2.35830243e+00, -1.91055713e+00, -4.81016962e-01,  3.42224532e-03,
         1.08802489e+00,  2.49587690e-01, -9.19107033e-01,  5.11694777e-01,
        -8.76958504e-01, -1.65785638e-01, -4.57655756e-01,  1.74897252e-01,
         7.90696465e-01,  2.02424375e-01,  1.04710158e+00, -7.82438092e-01,
       

In [21]:
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 5, eps=1e-6, use_indirect=True, scale=1)

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-06, alpha: 1.50, max_iters: 5000,
	  normalize: 1, scale: 1.00, adaptive_scaling: 1,
	  acceleration_lookback: 5, acceleration_interval: 50,
	  warm_start: 0
lin-sys:  sparse-indirect
	  nnz(A): 1500000, nnz(P): 500500, cg_rate: 2.00
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 5.40e+17  2.43e+20  1.25e+21 -9.73e+21  1.00e+00  1.30e+00 
   100| 4.04e-04  3.79e+00  6.33e+01 -4.81e+04  1.00e+00  1.35e+01 
   200| 1.35e-05  4.15e-03  4.02e-02 -4.81e+04  6.69e-02  1.83e+01 
   250| 4.81e-06  3.77e-04  4.7

{'x': array([-1.50004697e-01, -5.02997160e-01,  1.31843280e+00, -6.54430764e-01,
         2.80793055e-01, -8.16105163e-01, -4.24771370e-01,  6.75667986e-01,
         1.26249615e+00,  2.85813022e-01, -1.62501877e-01, -2.96832762e-01,
        -1.14297052e+00,  2.07967476e+00, -5.30664155e-01, -8.61221813e-01,
         4.62745208e-01, -1.71833325e+00, -1.13918944e+00,  2.64363925e-01,
        -1.00980906e+00,  1.54607040e-01,  5.29097531e-01, -2.39466272e-01,
         5.82330966e-01, -7.32734564e-01,  3.48466803e-01, -1.57540073e+00,
        -8.72642236e-01, -3.94820208e-01, -4.02691421e-01,  2.00434642e-01,
         8.95201546e-01,  1.18688789e+00, -9.53844721e-01, -3.52605981e-01,
        -2.35837450e+00, -1.91047704e+00, -4.81045150e-01,  3.56281991e-03,
         1.08797499e+00,  2.49517012e-01, -9.19236221e-01,  5.11691168e-01,
        -8.76947284e-01, -1.65604213e-01, -4.57638788e-01,  1.74862581e-01,
         7.90595877e-01,  2.02549508e-01,  1.04694375e+00, -7.82564406e-01,
       

In [22]:
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 0, eps=1e-6, use_indirect=True, scale=1)

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-06, alpha: 1.50, max_iters: 5000,
	  normalize: 1, scale: 1.00, adaptive_scaling: 1,
	  acceleration_lookback: 0, acceleration_interval: 50,
	  warm_start: 0
lin-sys:  sparse-indirect
	  nnz(A): 1500000, nnz(P): 500500, cg_rate: 2.00
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 5.40e+17  2.43e+20  1.25e+21 -9.73e+21  1.00e+00  1.21e+00 
   100| 4.67e-04  3.75e+00  6.45e+01 -4.81e+04  1.00e+00  1.28e+01 
   200| 1.45e-05  2.92e-03  4.71e-02 -4.81e+04  6.68e-02  1.78e+01 
   250| 4.91e-06  3.93e-04  2.1

{'x': array([-1.50005473e-01, -5.02992569e-01,  1.31843492e+00, -6.54429464e-01,
         2.80792126e-01, -8.16108199e-01, -4.24768964e-01,  6.75665792e-01,
         1.26248888e+00,  2.85810916e-01, -1.62508557e-01, -2.96840951e-01,
        -1.14296733e+00,  2.07967397e+00, -5.30667930e-01, -8.61225665e-01,
         4.62744118e-01, -1.71833195e+00, -1.13918292e+00,  2.64368913e-01,
        -1.00981155e+00,  1.54605866e-01,  5.29093759e-01, -2.39470572e-01,
         5.82335402e-01, -7.32731180e-01,  3.48464039e-01, -1.57539724e+00,
        -8.72643073e-01, -3.94821669e-01, -4.02682120e-01,  2.00431795e-01,
         8.95196658e-01,  1.18688780e+00, -9.53845471e-01, -3.52599447e-01,
        -2.35837374e+00, -1.91047737e+00, -4.81039387e-01,  3.56223755e-03,
         1.08797546e+00,  2.49509239e-01, -9.19234523e-01,  5.11690621e-01,
        -8.76952152e-01, -1.65612633e-01, -4.57632990e-01,  1.74862652e-01,
         7.90589792e-01,  2.02550707e-01,  1.04694039e+00, -7.82571755e-01,
       

In [23]:
np.random.seed(seed)
(P, A, b, c, _, _) = gen_feasible(m, n, p_scale=0.)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(x)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6)

probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6)

4999
[-2.77559638e-01 -4.80927966e-01  1.53034698e+00 -3.17972454e-01
  3.09092514e-01 -1.31256267e+00 -3.29024890e-01  5.24819776e-01
  7.83656168e-01  6.31902825e-04  4.36632194e-01 -8.63698806e-01
 -4.32571094e-01  1.60041909e+00 -4.45431707e-01 -8.38335587e-01
 -4.35391960e-02 -7.95475636e-01 -5.37570976e-01  7.01205239e-01
 -1.22726983e+00  2.74198451e-01  8.24613362e-01  3.72699894e-01
  1.23634856e-01 -7.09918384e-01  1.49955127e-03 -1.19238521e+00
 -8.10359843e-01  3.29492764e-01  2.97270940e-01  5.50337569e-01
  1.13137709e+00  9.41447827e-01 -8.35966364e-01 -1.33923765e-01
 -1.30011359e+00 -1.55445665e+00 -1.28543591e-01 -3.69934534e-01
  5.43098209e-01  1.87218598e-01 -4.87323784e-01  2.76246197e-01
 -9.57986046e-01 -4.40291675e-01 -6.88387743e-01  6.73758009e-01
  1.44046149e+00  1.63606865e-01  1.02404543e+00 -6.97729190e-01
  6.97724240e-01 -7.79347014e-01 -3.01293161e-01 -5.64765245e-01
  4.14367486e-02 -1.12158975e+00 -9.87546694e-03  7.15265893e-02
 -1.51303273e-01  5.

------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 3.19e+18  7.00e+17  4.82e+20 -5.01e+20  1.00e+00  2.19e-02 
   100| 4.96e-03  1.03e-03  2.52e-03  6.93e+00  1.00e+00  8.20e-01 
   200| 6.63e-04  2.29e-04  5.34e-03  6.94e+00  1.00e+00  1.63e+00 
   300| 1.28e-04  9.22e-05  2.08e-04  6.93e+00  1.00e+00  2.46e+00 
   400| 8.05e-05  5.72e-05  1.60e-03  6.94e+00  1.00e+00  3.29e+00 
   500| 3.08e-05  5.11e-05  1.07e-03  6.94e+00  1.00e+00  4.12e+00 
   600| 5.93e-05  2.32e-05  6.65e-05  6.94e+00  1.00e+00  4.94e+00 
   700| 1.75e-04  4.30e-05  5.67e-04  6.93e+00  1.00e+00  5.76e+00 
   800| 4.87e-05  1.81e-05  9.79e-05  6.94e+00  1.00e+00  6.60e+00 
   900| 5.99e-05  2.20e-05  7.81e-04  6.94e+00  1.00e+00  7.42e+00 
  1000| 1.02e-05  1.51e-05  7.77e-05  6.94e+00  1.00e+00  8.26e+00 
  1100| 1.06e-05  1.51e-05  7.95e-05  6.94e+00  1.0

{'x': array([-2.83386538e+00, -1.28971926e+01,  8.42337762e+00, -8.00483496e+00,
         9.44065675e-01, -5.75414727e+00,  8.47015973e+00, -2.50085978e+00,
         1.60931816e+00, -9.98492388e-01, -1.32911260e+01, -6.07047686e+00,
         4.08852274e+00,  4.06996692e+00,  2.36789585e+00,  5.39218681e+00,
        -3.36856912e+00, -1.18752694e+00,  1.36904959e+00, -4.30854538e+00,
        -1.01557419e+00,  3.10632173e+00, -1.00397469e+00, -7.32931853e-01,
        -8.16397327e-01, -2.35822296e+00,  2.74176728e+00, -8.06883700e+00,
        -7.06033447e+00,  2.79848634e+00, -7.88170274e-01,  6.17067315e-01,
         2.94451238e+00,  9.75286206e-01, -5.09406500e+00,  1.86657218e-01,
        -7.39933995e+00,  5.15858900e+00,  5.40657569e+00, -1.13983933e+00,
        -5.38131988e+00,  3.22946813e+00, -1.32640077e+00, -4.67160354e+00,
        -9.48782845e+00, -2.18796241e+00, -2.96384217e+00,  4.98793936e+00,
         1.39434395e+00, -2.18610757e+00,  4.55039009e+00,  9.71330961e+00,
       

In [24]:
sol = scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 0, eps=1e-6, use_indirect=True)
print(c.T @ sol['x'])

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-06, alpha: 1.50, max_iters: 5000,
	  normalize: 0, scale: 1.00, adaptive_scaling: 1,
	  acceleration_lookback: 0, acceleration_interval: 50,
	  warm_start: 0
lin-sys:  sparse-indirect
	  nnz(A): 1500000, nnz(P): 0, cg_rate: 2.00
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 5.43e+18  1.19e+18  6.33e+20 -6.63e+20  1.00e+00  5.35e+00 
   100| 6.35e-03  2.03e-03  1.95e-02  6.97e+00  1.00e+00  2.50e+01 
   200| 1.27e-03  5.25e-04  7.82e-03  6.93e+00  1.00e+00  4.30e+01 
   300| 5.11e-04  1.57e-04  8.86e-03

In [25]:
probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)q
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6, use_indirect=True)
print(c.T @ sol['x'])

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-06, alpha: 1.50, max_iters: 5000,
	  normalize: 1, scale: 1.00, adaptive_scaling: 1,
	  acceleration_lookback: 10, acceleration_interval: 50,
	  warm_start: 0
lin-sys:  sparse-indirect
	  nnz(A): 1500000, nnz(P): 0, cg_rate: 2.00
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 3.19e+18  7.00e+17  4.82e+20 -5.01e+20  1.00e+00  5.22e+00 
   100| 7.09e-03  1.82e-03  5.23e-02  6.89e+00  1.00e+00  2.83e+01 
   200| 1.73e-03  2.92e-04  7.73e-03  6.94e+00  1.00e+00  4.76e+01 
   300| 5.67e-04  1.81e-04  1.13e-0

In [26]:
np.random.seed(seed)
(P, A, b, c, _, _) = gen_feasible(m, n, p_scale=0.)
print(np.linalg.norm(A))

12.485287623108398


In [27]:
np.random.seed(seed)
(P, A, b, c, _, _) = gen_feasible(m, n, p_scale=0.)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(c.T @ x)
print(x)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6, use_indirect=True)

probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6, use_indirect=True)
print(c.T @ sol['x'])

4999
6.937899985531961
[-2.77559638e-01 -4.80927966e-01  1.53034698e+00 -3.17972454e-01
  3.09092514e-01 -1.31256267e+00 -3.29024890e-01  5.24819776e-01
  7.83656168e-01  6.31902825e-04  4.36632194e-01 -8.63698806e-01
 -4.32571094e-01  1.60041909e+00 -4.45431707e-01 -8.38335587e-01
 -4.35391960e-02 -7.95475636e-01 -5.37570976e-01  7.01205239e-01
 -1.22726983e+00  2.74198451e-01  8.24613362e-01  3.72699894e-01
  1.23634856e-01 -7.09918384e-01  1.49955127e-03 -1.19238521e+00
 -8.10359843e-01  3.29492764e-01  2.97270940e-01  5.50337569e-01
  1.13137709e+00  9.41447827e-01 -8.35966364e-01 -1.33923765e-01
 -1.30011359e+00 -1.55445665e+00 -1.28543591e-01 -3.69934534e-01
  5.43098209e-01  1.87218598e-01 -4.87323784e-01  2.76246197e-01
 -9.57986046e-01 -4.40291675e-01 -6.88387743e-01  6.73758009e-01
  1.44046149e+00  1.63606865e-01  1.02404543e+00 -6.97729190e-01
  6.97724240e-01 -7.79347014e-01 -3.01293161e-01 -5.64765245e-01
  4.14367486e-02 -1.12158975e+00 -9.87546694e-03  7.15265893e-02
 -

------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 3.19e+18  7.00e+17  4.82e+20 -5.01e+20  1.00e+00  5.29e+00 
   100| 7.09e-03  1.82e-03  5.23e-02  6.89e+00  1.00e+00  2.99e+01 
   200| 1.73e-03  2.92e-04  7.73e-03  6.94e+00  1.00e+00  5.15e+01 
   300| 5.67e-04  1.81e-04  1.13e-02  6.95e+00  1.00e+00  6.99e+01 
   400| 2.39e-04  7.56e-05  4.44e-03  6.94e+00  1.00e+00  8.89e+01 
   500| 1.14e-04  4.94e-05  4.04e-04  6.93e+00  1.00e+00  1.05e+02 
   600| 1.29e-04  5.55e-05  4.54e-03  6.94e+00  1.00e+00  1.27e+02 
   700| 1.49e-04  5.11e-05  2.77e-03  6.94e+00  1.00e+00  1.56e+02 
   800| 1.30e-04  5.04e-05  3.11e-03  6.94e+00  1.00e+00  1.88e+02 
   900| 8.60e-05  5.21e-05  5.29e-04  6.94e+00  1.00e+00  2.23e+02 
  1000| 5.93e-05  5.08e-05  1.49e-03  6.94e+00  1.00e+00  2.57e+02 
  1100| 5.56e-05  4.71e-05  1.84e-03  6.94e+00  1.0

  4600| 5.09e-06  1.48e-05  1.74e-04  6.94e+00  1.00e+00  2.03e+03 
  4700| 2.04e-05  1.64e-05  2.27e-04  6.94e+00  1.00e+00  2.09e+03 
  4800| 1.01e-05  1.61e-05  1.98e-04  6.94e+00  1.00e+00  2.16e+03 
  4900| 1.00e-05  1.61e-05  1.86e-04  6.94e+00  1.00e+00  2.22e+03 
  5000| 3.85e-05  1.79e-05  2.16e-04  6.94e+00  1.00e+00  2.29e+03 
------------------------------------------------------------------
hit max_iters, returning best iterate
status:  solved / inaccurate
timings: total: 2.29e+03s = setup: 1.23e-01s + solve: 2.29e+03s
	 lin-sys: 2.28e+03s, cones: 4.26e-02s, accel: 2.04e-02s
lin-sys: avg cg its: 110.98
------------------------------------------------------------------
cones: dist(s, K) = 0.00e+00, dist(y, K*) = 0.00e+00
comp slack: s'y/|s||y| = 0.00e+00, gap: |x'Px+c'x+b'y| = 9.09e-06
pri res: |Ax+s-b| = 1.41e-05, dua res: |Px+A'y+c| = 1.56e-05
------------------------------------------------------------------
optimal objective = 6.935417
----------------------------------